In [205]:
# Tree
def isLeaf(node): # ts
    if node.find('primary') is not None:
        return False
    else:
        return True

def getNote(node):
    return node.find('head').find('chord').find('note').attrib['id']
    
def xml2dict(node,lkup):
    obj = {'name':""}
    if not isLeaf(node):
        obj['children'] = []
        if getNote(node.find('primary').find('ts')) > getNote(node.find('secondary').find('ts')):
            obj['name'] = lkup[getNote(node)]
            obj['children'].append(xml2dict(node.find('secondary').find('ts'),lkup))
            obj['children'].append(xml2dict(node.find('primary').find('ts'),lkup))
        else:
            obj['name'] = lkup[getNote(node)]
            obj['children'].append(xml2dict(node.find('primary').find('ts'),lkup))
            obj['children'].append(xml2dict(node.find('secondary').find('ts'),lkup))
    else:
        obj['name'] = lkup[getNote(node)]
    return obj

def xml2dict_raw(node):
    obj = {'name':""}
    if not isLeaf(node):
        obj['children'] = []
        if getNote(node.find('primary').find('ts')) > getNote(node.find('secondary').find('ts')):
#             obj['name'] = lkup[getNote(node)]
            obj['children'].append(xml2dict_raw(node.find('secondary').find('ts')))
            obj['children'].append(xml2dict_raw(node.find('primary').find('ts')))
        else:
#             obj['name'] = lkup[getNote(node)]
            obj['children'].append(xml2dict_raw(node.find('primary').find('ts')))
            obj['children'].append(xml2dict_raw(node.find('secondary').find('ts')))
    else:
        obj['name'] = getNote(node)
    return obj

    

In [265]:
# DEBUG
# READ TS-XML
import xml.etree.ElementTree as ET
xmldoc = ET.parse('gttm/11/TS-11.xml')
root = xmldoc.getroot()
tree = []
tree.append(xml2dict_raw(root.find('ts')))
import json
json.dumps(tree)

'[{"children": [{"children": [{"children": [{"children": [{"children": [{"name": "P1-1-1"}, {"children": [{"name": "P1-2-1"}, {"children": [{"name": "P1-2-2"}, {"name": "P1-2-3"}], "name": ""}], "name": ""}], "name": ""}, {"children": [{"name": "P1-2-4"}, {"children": [{"name": "P1-2-5"}, {"name": "P1-2-6"}], "name": ""}], "name": ""}], "name": ""}, {"children": [{"children": [{"children": [{"name": "P1-3-1"}, {"name": "P1-3-2"}], "name": ""}, {"children": [{"name": "P1-3-3"}, {"name": "P1-3-4"}], "name": ""}], "name": ""}, {"name": "P1-3-5"}], "name": ""}], "name": ""}, {"children": [{"children": [{"children": [{"children": [{"children": [{"name": "P1-3-6"}, {"name": "P1-3-7"}], "name": ""}, {"name": "P1-4-1"}], "name": ""}, {"name": "P1-4-2"}], "name": ""}, {"children": [{"name": "P1-4-3"}, {"name": "P1-4-4"}], "name": ""}], "name": ""}, {"children": [{"name": "P1-4-5"}, {"name": "P1-4-6"}], "name": ""}], "name": ""}], "name": ""}, {"children": [{"children": [{"children": [{"children

In [259]:
# Dictionary
import xml.etree.ElementTree as ET
def getNotesFromXml(f):
    out = dict()
    root = ET.parse(f).getroot()
    ms = root.find('part').findall('measure')
    for m,m_el in enumerate(ms):
        for n,n_el in enumerate(m_el.findall('note')):
            if n_el.find('pitch') != None:
                acc = " "
                if n_el.find('accidental') != None:
                    if n_el.find('accidental').text == 'sharp':
                        acc = "+"
                    elif n_el.find('accidental').text == 'flat':
                        acc = "-"
                if n_el.find('pitch').find('alter') != None:
                    if n_el.find('pitch').find('alter').text == '1':
                        acc = "+"
                    elif n_el.find('pitch').find('alter').text == '-1':
                        acc = "-"
                out['P1-'+str(m_el.attrib['number'])+'-'+str(n+1)] = n_el.find('pitch').find('step').text + n_el.find('pitch').find('octave').text + acc + n_el.find('duration').text
    return out

In [260]:
# DEBUG
def getNotes(ii):
    return [getNotesFromXml(ii)[i] for i in sorted(list(getNotesFromXml(ii).keys()))]
#     print(getNotesFromXml('gttm/103/MSC-103.xml')[i])
# getNotesFromXml('gttm/103/MSC-103.xml')
getNotes('gttm/05/05_Turkish March.xml')

['F5 2',
 'D5 2',
 'D5 2',
 'D5 2',
 'F5 2',
 'D5 2',
 'D5 2',
 'D6 2',
 'C6 1',
 'B5-1',
 'A5 1',
 'G5 1',
 'F5 2',
 'E5-2',
 'D5 2',
 'E5-2',
 'F5 4',
 'F5 2',
 'D5 2',
 'D5 2',
 'D5 2',
 'F5 2',
 'D5 2',
 'D5 2',
 'D6 2',
 'C6 1',
 'B5-1',
 'A5 1',
 'G5 1',
 'F5 2',
 'A5 2',
 'B5-2']

In [261]:
# Forest, one giant json of tree
import os
import json
ex = ['GP','PR','HM','TS','MP','GR']
trees = []
fo = open("tree.json", "wb")
fo.write("var tree = [".encode('utf-8'))
for x in range(1,268):
    lkup = dict()
    for file in os.listdir("gttm/"+str(x).zfill(2)):
        
        if file.endswith(".xml") and file[:2] not in ex:
            print("gttm/"+str(x).zfill(2)+"/"+file)
            lkup = getNotesFromXml("gttm/"+str(x).zfill(2)+"/"+file)
#             print(lkup)
    for file in os.listdir("gttm/"+str(x).zfill(2)):
        if file.endswith(".xml") and file[:2] == "TS":
            print("gttm/"+str(x).zfill(2)+"/"+file)
            # Open a file
            
            xmldoc = ET.parse("gttm/"+str(x).zfill(2)+"/"+file)
            root = xmldoc.getroot()
            tree = []
            tree.append(xml2dict(root.find('ts'),lkup))
#             print(tree[0])
            fo.write(json.dumps(tree[0]).encode('utf-8'));
            if x!=267:
                fo.write(','.encode('utf-8'));
            # Close opend file
fo.write("]".encode('utf-8'))
fo.close()
            

gttm/01/01_Waltz in E flat Grande Valse Brillante Op.18.xml
gttm/01/TS-01.xml
gttm/02/02_Moments Musicaux.xml
gttm/02/TS-02.xml
gttm/03/03_Bagatelle 'Fur Elise' WoO.59.xml
gttm/03/TS-03.xml
gttm/04/04_The Preludes Op.28 No.15.xml
gttm/04/TS-04.xml
gttm/05/05_Turkish March.xml
gttm/05/TS-05.xml
gttm/06/06_Blumenlied Op.39.xml
gttm/06/TS-06.xml
gttm/07/07_Nocturne.xml
gttm/07/TS-07.xml
gttm/08/08_Spinnerlied Op.14 No.4.xml
gttm/08/TS-08.xml
gttm/09/09_String Quartet in F major Op.3 No.5 Serenade.xml
gttm/09/TS-09.xml
gttm/10/10_Wiegenlied.xml
gttm/10/TS-10.xml
gttm/11/11_Solvejgs Lied.xml
gttm/11/TS-11.xml
gttm/12/12_Anitras Dans.xml
gttm/12/TS-12.xml
gttm/13/13_Traumerei.xml
gttm/13/TS-13.xml
gttm/14/14_Menuett No.2 in G maj.xml
gttm/14/TS-14.xml
gttm/15/15_Aida Fernando Corena.xml
gttm/15/TS-15.xml
gttm/16/16_William Tell Overture.xml
gttm/16/TS-16.xml
gttm/17/17_Carmen.xml
gttm/17/TS-17.xml
gttm/18/18_Thais Meditation.xml
gttm/18/TS-18.xml
gttm/19/19_Tannhauser Overture.xml
gttm/19/TS

In [263]:
# notes, list of pitch and rythm
import os
import json
ex = ['GP','PR','HM','TS','MP','GR']
trees = []
fo = open("tree.json", "wb")
fo.write("var noteList = [".encode('utf-8'))
for x in range(1,268):
# if True:
    lkup = dict()
    for file in os.listdir("gttm/"+str(x).zfill(2)):
        if file.endswith(".xml") and file[:2] not in ex:
            print("gttm/"+str(x).zfill(2)+"/"+file)
#             print(getNotes("gttm/"+str(x).zfill(2)+"/"+file))
            fo.write(json.dumps(getNotes("gttm/"+str(x).zfill(2)+"/"+file)).encode('utf-8'));
            if x!=267:
                fo.write(','.encode('utf-8'));
            # Close opend file
fo.write("]".encode('utf-8'))
fo.close()
            

gttm/01/01_Waltz in E flat Grande Valse Brillante Op.18.xml
gttm/02/02_Moments Musicaux.xml
gttm/03/03_Bagatelle 'Fur Elise' WoO.59.xml
gttm/04/04_The Preludes Op.28 No.15.xml
gttm/05/05_Turkish March.xml
gttm/06/06_Blumenlied Op.39.xml
gttm/07/07_Nocturne.xml
gttm/08/08_Spinnerlied Op.14 No.4.xml
gttm/09/09_String Quartet in F major Op.3 No.5 Serenade.xml
gttm/10/10_Wiegenlied.xml
gttm/11/11_Solvejgs Lied.xml
gttm/12/12_Anitras Dans.xml
gttm/13/13_Traumerei.xml
gttm/14/14_Menuett No.2 in G maj.xml
gttm/15/15_Aida Fernando Corena.xml
gttm/16/16_William Tell Overture.xml
gttm/17/17_Carmen.xml
gttm/18/18_Thais Meditation.xml
gttm/19/19_Tannhauser Overture.xml
gttm/20/20_La Traviata Brindisi.xml
gttm/21/21_Plaisir d'Amour.xml
gttm/22/22_Ellens Gesang III Op.52-6 D.839.xml
gttm/23/23_Die Meistersinger von Nurnberg Prelude.xml
gttm/24/24_Orphee aux Enfers Overture.xml
gttm/25/25_L'arlesienne Suite No.2 Farandole.xml
gttm/26/26_Overture No.3 BWV.1068 Air on a G string.xml
gttm/27/27_Peer Gyn